In [47]:
%matplotlib inline
import random
import ast
from operator import itemgetter
import numpy as np
from multiprocessing.pool import ThreadPool

from pyquil.quil import Program, Pragma, Program
from pyquil.api import QVMConnection, CompilerConnection, get_devices

In [ ]:
import os

API_KEY = 'nmRPAVunQl19TtQz9eMd11iiIsArtUDTaEnsSV6u'
USER_ID = '221422a5-07ab-4cb2-8d4d-2b3dae3aedc7'

PYQUIL_CONFIG = f"""
[Rigetti Forest]
url: https://api.rigetti.com/qvm
key: {API_KEY}
user_id: {USER_ID}
"""

with open(os.path.expanduser('~/.pyquil_config'), 'w') as f:
    f.write(PYQUIL_CONFIG)

In [3]:
def all_same(items):
    return all(x == items[0] for x in items)

def tester(bitstring,n=2,m=2):
   # print(np.array(bitstring))
    mySplit = np.split(np.array(bitstring),n)
    for line in mySplit:
        if not all_same(line):
            mySplit = np.transpose(mySplit)
            for col in mySplit:
                if not all_same(col):
                    return False
            return True
    return True

In [15]:
#This class can be used to optimize a quantum circuit for any target probability distribution. 
class DDQCL():
    def __init__(self, init_depth=6, num_qubits=4):
        self.gate_dict={0:['X', 1], 1:['Y', 1], 2:['H', 1], 3:['Z', 1], 4:['CNOT', 2], 5:['ISWAP', 2]}
        self.init_depth = init_depth
        self.chrom_len = 3
        self.num_gates = len(self.gate_dict)
        self.num_qubits = num_qubits
        
        self.best_gene = []
        self.best_fitness = 0
    
    def program_constructor(self, gene):        
        '''Takes in the gene as a parameter and returns the quantum circuit the gene codes for'''
        program_string=''
        for chrom in gene:
            gate_info = self.gate_dict[chrom[0]]
            program_string+= gate_info[0]+ ''.join([' '+str(qubit) for qubit in chrom[1:gate_info[1]+1]]) + ' \n'
        program_string+='MEASURE 0[0] \nMEASURE 1[1] \nMEASURE 2[2] \nMEASURE 3[3]'
        return program_string
    
    #crossover operation between two parent genes
    def crossover(self, gene1, gene2):
        '''Created a combined gene by splicing the two parent genes together
        and samples from it to create the child gene'''
        child = []
        maxlen = max(len(gene1), len(gene2))
        combined = []
        for k in range(maxlen):
            if k<len(gene1):
                combined.append(gene1[k])
            if k<len(gene2):
                combined.append(gene2[k])

        for chrom in combined:
            prob_sample = random.uniform(0, 1)
            if prob_sample>=0.5:
                child.append(chrom)
        if len(child)==0:
            child.append(gene1[0])
        return child
    
    def breed(self, breed_percent):
        '''Replace bottom breed_percent% of the population with children created
        by performing crossover operation on best performing candidates' genes '''
        number_replace = int(len(self.population_genes)*breed_percent)
        for k in range(number_replace):
            parents = random.sample(self.population_genes[number_replace:], 2)
            child = self.crossover(parents[0], parents[1])
            self.population_genes[k] = child

    def mutate(self, mutation_prob=0.1):
        '''Add noise to each candidate's gene in the population'''
        for i, gene in enumerate(self.population_genes):
            for j, chrom in enumerate(gene):
                for k, genotype in enumerate(chrom):
                    prob_sample = random.uniform(0, 1)
                    if prob_sample<mutation_prob:
                        if k==0:
                            self.population_genes[i][j][k] = random.randint(0, self.num_gates-1)
                        else:
                            self.population_genes[i][j][k] = random.randint(0, self.num_qubits-1)
                            #ensure three unique qubits for operations
                            qubitset = set(self.population_genes[i][j][1:]) 
                            while len(qubitset) != self.chrom_len-1: #or qubitset==set((0, 2)) or qubitset==set((0, 3)) or qubitset==set((1, 3)): 
                                self.population_genes[i][j][k] = random.randint(0, self.num_qubits-1)
                                qubitset = set(self.population_genes[i][j][1:])
                            break
    
    def evaluate_fitness(self, datinput, trials=1000):
        """
        Thin function that takes a program string and returns the F1 score on the Bars and Stripes task
        """
        i, quil_program = datinput
        qvm = QVMConnection()
        results = qvm.run(Program(quil_program), trials=trials)
        results = list(map(tuple, results))

        observed_results = set(results)
        recall = 0
        precision = 0
        for result in sorted(observed_results):
            bitstring = ''.join(reversed(list(map(str, result))))
            failer = tester(list(map(str, result)))
            p_i = results.count(result)/len(results) +0.0001
            if failer:
                recall+=1
                precision += p_i
                
            #print(str(failer) + f'|{bitstring}> state: {p_i} [{results.count(result)}/{len(results)}]')
        recall /= 6
        f1 = (2*precision*recall)/(precision+recall+0.0001)
        #print(f1)
        self.fitnesses[i] = f1
        return f1
        
    def run(self, pop_size=50, gens=5, n_top=5):
        '''Perform the optimization using the rest of the functions defined in this class'''
        #initialize population
        self.population_genes = []
        self.fitnesses = [0]*pop_size
        
        #initialize the population with random genes
        for i in range(pop_size):
            depth = random.randint(3, self.init_depth)
            gene = []
            for j in range(depth):
                chrom = []
                chrom.append(random.randint(0, self.num_gates-1))
                for k in range(self.chrom_len-1):
                    chrom.append(random.randint(0, self.num_qubits-1))
                
                qubitset = set(chrom[1:])
                while len(qubitset) != self.chrom_len-1:# or qubitset==set((0, 2)) or qubitset==set((0, 3)) or qubitset==set((1, 3)): 
                    chrom[random.randint(1, self.chrom_len-1)] = random.randint(0, self.num_qubits-1)
                    qubitset = set(chrom[1:])
                gene.append(chrom)
            self.population_genes.append(gene)
        #for each generation
        for g in range(gens):
            #evaluate fitness of population
            '''for k, gene in enumerate(self.population_genes):
                program_string = self.program_constructor(gene)
                fitness = self.evaluate_fitness(program_string)
                fitnesses[k] = fitness'''
            
            programs = []
            for k, gene in enumerate(self.population_genes):
                program_string = self.program_constructor(gene)
                programs.append(program_string)
            
            with ThreadPool(32) as p:
                results = p.map(self.evaluate_fitness, list(enumerate(programs)))
                p.close()
                p.join()

            #sort population_genes by fitness
            self.population_genes, self.fitnesses = zip(*sorted(list(zip(self.population_genes, self.fitnesses)), key=itemgetter(1)))
            self.population_genes, self.fitnesses = list(self.population_genes), list(self.fitnesses)

            #print generation stats
            print('Generation ', g+1 , ' top ', n_top, ' architectures: ')
            for gene, fitness in zip(self.population_genes[-n_top:], self.fitnesses[-n_top:]):
                print('Gene: ', gene, ' Fitness: ', fitness)
            print()
            '''print('Worst: ', self.population_genes[0], 'Fitness: ', self.fitnesses[0])
            print()'''
            
            with open('Results.txt', 'a+') as f:
                f.write('Gene: '+str(self.population_genes[-1])+' Fitness: '+str(self.fitnesses[-1])+'\n\n')
                
            if self.fitnesses[-1]>self.best_fitness:
                self.best_fitness = self.fitnesses[-1]
                self.best_gene = self.population_genes[-1]
            
            if g!=(gens-1): #no need to breed and mutate on final generation
                
                #breed to replace least fit members of population
                self.breed(breed_percent=0.7)
                
                #mutate all members
                self.mutate(mutation_prob=0.1)
                
                '''for gene, fitness in zip(self.population_genes[-n_top:], self.fitnesses[-n_top:]):
                    print('Gene: ', gene, ' Fitness: ', fitness)
                print()
                print('Worst: ', self.population_genes[0], 'Fitness: ', self.fitnesses[0])'''

        with open('Results.txt', 'a+') as f:
            f.write('Best Gene: '+str(self.best_gene)+' Fitness: '+str(self.best_fitness)+'\n\n')

In [16]:
learner = DDQCL(init_depth=6)
#gene1 = [[0, 2, 1, 3], [4, 1, 2, 3], [3, 3, 2, 1]]
#gene2 = [[1, 3, 2, 0], [2, 2, 3, 1], [3, 3, 2, 0]]
'''print(learner.program_constructor(gene1))
print(learner.program_constructor(gene2))

child = learner.crossover(gene1, gene2)
print()
print(learner.program_constructor(child))'''

learner.run(pop_size=2000, gens=1000)

Generation  1  top  5  architectures: 
Gene:  [[5, 2, 3], [2, 3, 0], [4, 3, 2], [4, 0, 3]]  Fitness:  0.4999875028118673
Gene:  [[5, 0, 1], [4, 2, 3], [3, 0, 1], [3, 2, 1], [2, 2, 1], [4, 2, 0]]  Fitness:  0.4999875028118673
Gene:  [[0, 3, 1], [2, 2, 3], [4, 2, 3], [0, 2, 1], [3, 0, 3]]  Fitness:  0.4999875028118673
Gene:  [[1, 2, 1], [2, 1, 2], [0, 2, 3], [4, 1, 0], [4, 3, 0]]  Fitness:  0.4999875028118673
Gene:  [[5, 0, 3], [2, 3, 2], [2, 0, 1], [2, 1, 3], [4, 0, 2]]  Fitness:  0.5806688302525751

Generation  2  top  5  architectures: 
Gene:  [[4, 3, 2], [2, 2, 0], [1, 1, 0], [2, 0, 2], [2, 3, 2], [2, 1, 0]]  Fitness:  0.5343298893529713
Gene:  [[2, 0, 3], [1, 2, 0], [2, 1, 0], [2, 3, 0], [4, 3, 2], [2, 2, 3]]  Fitness:  0.5407456044356898
Gene:  [[2, 2, 1], [3, 0, 3], [2, 1, 0], [2, 0, 1], [2, 3, 2], [3, 1, 0]]  Fitness:  0.5637969411933654
Gene:  [[2, 2, 0], [0, 3, 2], [2, 0, 2], [4, 0, 1], [2, 3, 2]]  Fitness:  0.5761816891412349
Gene:  [[5, 0, 3], [2, 3, 0], [2, 0, 1], [2, 1, 3],

Generation  18  top  5  architectures: 
Gene:  [[5, 3, 0], [4, 3, 0], [3, 0, 2]]  Fitness:  0.285693881048963
Gene:  [[4, 3, 0], [3, 0, 2]]  Fitness:  0.285693881048963
Gene:  [[1, 3, 2], [1, 3, 2], [2, 0, 1], [2, 1, 0]]  Fitness:  0.3886781441494018
Gene:  [[2, 1, 0], [2, 0, 3], [5, 3, 0], [5, 3, 0]]  Fitness:  0.39808288694671556
Gene:  [[3, 2, 3], [2, 0, 1], [2, 1, 0]]  Fitness:  0.40868118689311

Generation  19  top  5  architectures: 
Gene:  [[4, 1, 2], [2, 1, 2], [2, 3, 2], [0, 3, 2]]  Fitness:  0.40001599424207285
Gene:  [[2, 2, 1], [2, 0, 1]]  Fitness:  0.4009721502848719
Gene:  [[2, 0, 1], [2, 1, 3]]  Fitness:  0.40128934696963664
Gene:  [[2, 1, 3], [2, 3, 0]]  Fitness:  0.40317673554861905
Gene:  [[2, 1, 3], [2, 0, 1]]  Fitness:  0.4095763136589691

Generation  20  top  5  architectures: 
Gene:  [[2, 2, 0], [0, 3, 2], [2, 1, 0], [3, 0, 1], [2, 3, 2], [0, 3, 2]]  Fitness:  0.40984186924710886
Gene:  [[2, 2, 0], [2, 1, 0], [2, 3, 2], [0, 3, 2]]  Fitness:  0.4274101213647813
Gen

Generation  39  top  5  architectures: 
Gene:  [[2, 2, 1], [4, 2, 0]]  Fitness:  0.4999875028118673
Gene:  [[4, 1, 3], [2, 1, 0], [4, 1, 3]]  Fitness:  0.4999875028118673
Gene:  [[2, 1, 2], [4, 1, 3]]  Fitness:  0.4999875028118673
Gene:  [[2, 2, 1], [4, 2, 0]]  Fitness:  0.4999875028118673
Gene:  [[2, 2, 1], [4, 2, 0]]  Fitness:  0.4999875028118673

Generation  40  top  5  architectures: 
Gene:  [[5, 1, 0], [2, 0, 2], [5, 1, 0], [2, 0, 2]]  Fitness:  0.40411039804716714
Gene:  [[3, 3, 2], [2, 1, 2], [2, 0, 2], [3, 3, 2]]  Fitness:  0.4044201502221873
Gene:  [[2, 1, 2], [5, 1, 0], [2, 1, 2]]  Fitness:  0.4044201502221873
Gene:  [[5, 1, 0], [2, 0, 2], [5, 1, 0], [2, 0, 2]]  Fitness:  0.4065681188813916
Gene:  [[5, 2, 0], [2, 0, 1], [5, 2, 0], [2, 0, 1]]  Fitness:  0.41222480827196767

Generation  41  top  5  architectures: 
Gene:  [[2, 3, 2], [0, 2, 3], [1, 2, 1], [4, 3, 2]]  Fitness:  0.4999875028118673
Gene:  [[2, 3, 2], [4, 3, 2]]  Fitness:  0.4999875028118673
Gene:  [[2, 3, 2], [4, 3

Generation  60  top  5  architectures: 
Gene:  [[4, 3, 1], [5, 1, 3]]  Fitness:  0.285693881048963
Gene:  [[4, 1, 0], [2, 1, 0], [4, 1, 0], [2, 1, 0]]  Fitness:  0.39479799421093403
Gene:  [[2, 1, 0], [3, 0, 3], [4, 1, 0], [2, 1, 0]]  Fitness:  0.3977579741118593
Gene:  [[2, 1, 0], [4, 1, 0], [2, 1, 0], [4, 1, 0]]  Fitness:  0.4009721502848719
Gene:  [[2, 1, 0], [5, 3, 2], [5, 3, 2], [4, 1, 0]]  Fitness:  0.4999875028118673

Generation  61  top  5  architectures: 
Gene:  [[2, 0, 1], [0, 2, 1], [2, 0, 1], [0, 2, 1]]  Fitness:  0.285693881048963
Gene:  [[0, 2, 1], [2, 0, 1], [0, 2, 1], [2, 0, 1]]  Fitness:  0.285693881048963
Gene:  [[2, 3, 2], [0, 2, 1], [2, 0, 1]]  Fitness:  0.3974322742137349
Gene:  [[2, 3, 2], [4, 3, 2], [2, 3, 2], [4, 3, 2]]  Fitness:  0.405037467142689
Gene:  [[2, 0, 1], [4, 0, 2], [2, 0, 1]]  Fitness:  0.40868118689311

Generation  62  top  5  architectures: 
Gene:  [[2, 0, 1], [0, 2, 3], [0, 2, 3], [2, 2, 1]]  Fitness:  0.39379680019640734
Gene:  [[2, 0, 1], [2, 2

Generation  80  top  5  architectures: 
Gene:  [[2, 0, 1], [5, 0, 1], [2, 0, 1], [0, 1, 0]]  Fitness:  0.39873036281088037
Gene:  [[0, 2, 3], [0, 2, 3], [2, 0, 1], [0, 2, 3], [0, 2, 3], [2, 1, 3]]  Fitness:  0.39969574442531725
Gene:  [[0, 2, 3], [2, 0, 1], [0, 2, 3], [2, 1, 3]]  Fitness:  0.4022364090566636
Gene:  [[0, 1, 0], [2, 0, 1], [5, 0, 1], [2, 0, 1]]  Fitness:  0.40348869331859977
Gene:  [[2, 0, 1], [5, 0, 1], [2, 0, 1], [0, 1, 0]]  Fitness:  0.4044201502221873

Generation  81  top  5  architectures: 
Gene:  [[3, 0, 3], [4, 1, 2], [3, 0, 3], [3, 1, 3]]  Fitness:  0.285693881048963
Gene:  [[4, 3, 0], [3, 0, 3], [4, 3, 0], [3, 0, 3]]  Fitness:  0.285693881048963
Gene:  [[4, 3, 0], [5, 0, 2], [4, 3, 0], [3, 0, 3]]  Fitness:  0.285693881048963
Gene:  [[3, 0, 3], [4, 3, 0], [5, 0, 2], [4, 3, 0]]  Fitness:  0.285693881048963
Gene:  [[4, 3, 0], [5, 0, 2], [4, 3, 0], [3, 0, 3]]  Fitness:  0.285693881048963

Generation  82  top  5  architectures: 
Gene:  [[5, 1, 3], [2, 3, 2], [5, 1, 3

KeyboardInterrupt: 

In [10]:
best_program = learner.program_constructor([[1, 0, 2], [2, 3, 0], [3, 1, 2], [2, 1, 3], [5, 0, 1], [5, 1, 0], [4, 1, 2], [0, 1, 2], [2, 0, 2], [3, 3, 0], [4, 3, 1], [4, 0, 1], [1, 1, 0]])
print(best_program)

Y 0 
H 3 
Z 1 
H 1 
ISWAP 0 1 
ISWAP 1 0 
CNOT 1 2 
X 1 
H 0 
Z 3 
CNOT 3 1 
CNOT 0 1 
Y 1 
MEASURE 0[0] 
MEASURE 1[1] 
MEASURE 2[2] 
MEASURE 3[3]


In [13]:
def execute_alt(quil_program, trials=1000, silent=False, raw=False):
    """
    Thin function that takes a low-level Quil program and returns the
    resulting probability distribution.
    """

    qvm = QVMConnection()
    results = qvm.run(Program(quil_program), trials=trials)
    results = list(map(tuple, results))

    if not silent:
        recall = 0
        precision = 0
        observed_results = set(results)
        for result in sorted(observed_results):
            bitstring = ''.join(reversed(list(map(str, result))))
            failer = tester(list(map(str, result)))
            p_i = results.count(result)/len(results) +0.0001
            if failer:
                recall+=1
                precision += p_i
            print(f'|{bitstring}> state: {results.count(result)/len(results)} [{results.count(result)}/{len(results)}]')
        if raw:
            print(f'Results: {results}')
        recall /= 6
        f1 = (2*precision*recall)/(precision+recall+0.0001)
        print('F1 score: ', f1)

In [14]:
execute_alt(best_program)

|0000> state: 0.118 [118/1000]
|1100> state: 0.135 [135/1000]
|1010> state: 0.132 [132/1000]
|0110> state: 0.131 [131/1000]
|1001> state: 0.127 [127/1000]
|0101> state: 0.104 [104/1000]
|0011> state: 0.123 [123/1000]
|1111> state: 0.13 [130/1000]
F1 score:  0.8522407758076548


19 tot 4 double: [[3, 1, 0], [2, 0, 3], [0, 1, 3], [2, 3, 2], [0, 1, 3], [0, 3, 0], [3, 0, 1], [5, 2, 1], [4, 0, 1], [1, 1, 0], [2, 2, 3], [1, 3, 0], [4, 3, 1], [3, 3, 1], [1, 1, 0], [0, 3, 0], [1, 3, 2], [3, 0, 2], [4, 2, 3]]

13 tot 5 double: [[1, 0, 2], [2, 3, 0], [3, 1, 2], [2, 1, 3], [5, 0, 1], [5, 1, 0], [4, 1, 2], [0, 1, 2], [2, 0, 2], [3, 3, 0], [4, 3, 1], [4, 0, 1], [1, 1, 0]]

In [77]:
def gate_compiler(rig_Code):
    
    devices = get_devices(as_dict=True)
    agave = devices['8Q-Agave']
    compiler = CompilerConnection(agave)

    job_id = compiler.compile_async(Program(learner.program_constructor(ast.literal_eval(rig_Code.strip()))))
    job = compiler.wait_for_job(job_id)
    
    print('compiled quil', job.compiled_quil())
    return (job.gate_depth(),job.program_fidelity())

In [50]:
def splitfitness(thestring):
    [genome, score] = thestring.split('Fitness: ')
    genome = genome[6:-1]
    return [genome, score]

with open('Results.txt', 'r') as f:
    txt = f.read()
splittxt = txt.split('\n\n')

In [71]:
alltogether = list(map(splitfitness, splittxt))
sortedall = sorted(alltogether, key=lambda x:x[1], reverse=True)

In [72]:
infoline = list(map(gate_compiler, list(zip(*sortedall))[0]))

job SytNvhzQKrpNhzQe is currently compiling
job shVNiKnLLemgzEnx is currently compiling
job DJTKGiOlfKGihehT is currently compiling
job hXJzyBFPPsDwXJxy is currently compiling
job vRwUoqioifWKGfkJ is currently compiling
job cEpKKkbpDjTZbgzg is currently compiling
job SnQIxVUtpidiEcaA is currently compiling
job ImrvOGCBSHgrMrTH is currently compiling
job kCVTnaeESQlmuyjd is currently compiling
job XIPrLRWXkKjCZfHE is currently compiling
job ivavzMlmmOvZjdnw is currently compiling
job CZJrmYjqsZFWSkmk is currently compiling
job nJowiMGHwxHusRPE is currently compiling
job ASacWTscZifAwzPX is currently compiling
job IJVTGMbzkCKOsGNZ is currently compiling
job HfggubrduyQhxIuW is currently compiling
job BgkGCrFjvhNmYsRL is currently compiling
job grnQWlNaPAltaCUM is currently compiling
job GkJHUAlMRNlaIPqu is currently queued for compilation
job vLtYlUKrWAVbWfRX is currently compiling
job uQeOjMwcSnCNqOdW is currently compiling
job LYthGQAxWcDRvCny is currently compiling
job TkudIxWaQkCCHPO

In [73]:
allinfo = list(zip(infoline, list(zip(*sortedall))[1]))

In [74]:
for i, item in enumerate(allinfo):
    print(i, item)

0 ((100, 0.0), '0.8836300719986605')
1 ((125, 0.0), '0.8836300719986605')
2 ((236, 0.0), '0.8836300719986605')
3 ((144, 0.0), '0.8830066454459151')
4 ((174, 0.0), '0.8823825222104262')
5 ((122, 0.0), '0.8817577011237211')
6 ((160, 0.0), '0.8805059607096882')
7 ((189, 0.0), '0.8805059607096882')
8 ((75, 0.0), '0.8805059607096882')
9 ((91, 0.0), '0.8792514148036085')
10 ((92, 0.0), '0.8792514148036085')
11 ((171, 0.0), '0.8792514148036085')
12 ((263, 0.0), '0.8792514148036085')
13 ((182, 0.0), '0.8786230868419578')
14 ((211, 0.0), '0.8786230868419578')
15 ((153, 0.0), '0.8779940539630897')
16 ((127, 0.0), '0.8779940539630897')
17 ((143, 0.0), '0.8779940539630897')
18 ((233, 0.0), '0.8779940539630897')
19 ((137, 0.0), '0.8779940539630897')
20 ((173, 0.0), '0.8779940539630897')
21 ((214, 0.0), '0.8773643149800752')
22 ((135, 0.0), '0.8773643149800752')
23 ((241, 0.0), '0.8773643149800752')
24 ((251, 0.0), '0.8773643149800752')
25 ((88, 0.0), '0.8767338687033189')
26 ((123, 0.0), '0.8767338

In [87]:
print(learner.program_constructor(ast.literal_eval(list(zip(*sortedall))[0][131])))

Y 0 
H 3 
Z 1 
H 1 
ISWAP 0 1 
ISWAP 1 0 
CNOT 1 2 
X 1 
H 0 
Z 3 
CNOT 3 1 
CNOT 0 1 
Y 1 
MEASURE 0[0] 
MEASURE 1[1] 
MEASURE 2[2] 
MEASURE 3[3]


In [88]:
gate_compiler(list(zip(*sortedall))[0][131])

job MXsHaDkSxkurDnWY is currently compiling
compiled quil PRAGMA EXPECTED_REWIRING "#(0 1 2 3 4 5 6 7)"
RZ(0.5474521065004054) 0
RX(pi/2) 0
RZ(1.4093738333711365) 0
RX(-pi/2) 0
RZ(-1.8798809706524215) 0
RZ(2.0783973416692327) 1
RX(pi/2) 1
RZ(0.9128059314825487) 1
RX(-pi/2) 1
CZ 0 1
RZ(pi/2) 0
RX(-pi) 0
CZ 0 1
RZ(-1.0537901828308993) 3
RX(pi/2) 3
RZ(1.7571084916166457) 3
RX(-pi/2) 3
RZ(0.8653450274961036) 3
RZ(-2.1376648670344) 0
RX(pi/2) 0
RZ(0.5684189363814549) 0
RX(-pi/2) 0
RZ(-1.2675983656591754) 0
RZ(1.2429322797929623) 1
RX(pi/2) 1
RZ(1.965569971524997) 1
RX(-pi/2) 1
RZ(-pi/2) 2
RX(pi/2) 2
CZ 2 1
RZ(1.674236967864048) 2
RX(pi/2) 2
RZ(1.4189783790674746) 2
RX(-pi/2) 2
CZ 3 2
RZ(1.6366529270088535) 2
RX(pi/2) 2
RZ(-pi/2) 3
RX(-pi/2) 3
CZ 3 2
RX(-pi/2) 2
RX(pi/2) 3
CZ 3 2
RZ(-0.7241298561798137) 1
RX(pi/2) 1
RZ(-0.6542456812873576) 2
RX(pi/2) 2
RZ(0.9242262418970197) 2
RX(-pi/2) 2
CZ 1 2
CZ 1 0
RZ(pi) 0
RZ(pi) 1
RX(pi/2) 1
RZ(1.5707963267948968) 1
RZ(0.6463506146747173) 2
RZ(1.456437

(33, 4.204944854941015e-05)

In [89]:
execute_alt("""
Y 0 
H 3 
Z 1 
H 1 
ISWAP 0 1 
ISWAP 1 0 
CNOT 1 2 
X 1 
H 0 
Z 3 
CNOT 3 1 
CNOT 0 1 
Y 1 
MEASURE 0[0] 
MEASURE 1[1] 
MEASURE 2[2] 
MEASURE 3[3]
""")

|0000> state: 0.118 [118/1000]
|1100> state: 0.122 [122/1000]
|1010> state: 0.136 [136/1000]
|0110> state: 0.118 [118/1000]
|1001> state: 0.134 [134/1000]
|0101> state: 0.118 [118/1000]
|0011> state: 0.135 [135/1000]
|1111> state: 0.119 [119/1000]
F1 score:  0.8561788757362612
